# Import der Daten für den Cube

Liest die Exceldatei der Übung *University* in eine SQL Server Datenbank, sodass sie mit den Analysis Services weiter verarbeitet werden kann.
Es muss ein SQL Server verwendet werden, auf dem die multidimensional analysis services laufen.
Der Docker Container von SQL Server hat diesen Dienst nicht!

Quelle: http://griesmayer.com/?menu=Business%20Intelligence&semester=Semsester_6&topic=03_Cube

In [1]:
from pathlib import Path
import sqlalchemy, re
import pandas as pd

# Prompt for IP, username and password.
host = input("Host (IP), Enter für localhost.") or "localhost" if "host" not in locals() else host
username = input("Username, Enter für sa.") or "sa" if "username" not in locals() else username
password = input("Passwort, Enter für SqlServer2019.") or "SqlServer2019" if "password" not in locals() else password
database = "University"
connection_url = sqlalchemy.engine.URL.create("mssql+pyodbc", username=username,
    password=password, host=host, database=database,
    query={ "driver": "ODBC Driver 18 for SQL Server" })
# We cannot connect to sales to create the database (does not exist at this time). We use tempdb.
# Autocommit is necessary for create database and ddl statements.
tempdb_engine = sqlalchemy.create_engine(
    connection_url.set(database="tempdb"), isolation_level="AUTOCOMMIT", 
    connect_args={"TrustServerCertificate": "yes"})
# We drop the database just before connecting, so we set pool_pre_ping=True
engine = sqlalchemy.create_engine(
    connection_url, fast_executemany=True, pool_pre_ping=True,
    connect_args={"TrustServerCertificate": "yes"})


Zuerst löschen wir die Datenbank und erstellen sie neu.
Das ist natürlich nur zum Testen, sonst ist das Löschen der Datenbank nicht ideal...

In [2]:
with tempdb_engine.connect() as conn: 
    try: conn.execute(sqlalchemy.text(f"ALTER DATABASE {database} SET SINGLE_USER WITH ROLLBACK IMMEDIATE"))
    except: pass
    conn.execute(sqlalchemy.text(f"DROP DATABASE IF EXISTS {database}"))
    conn.execute(sqlalchemy.text(f"CREATE DATABASE {database}"))
with engine.connect() as conn:
    conn.execution_options(isolation_level="AUTOCOMMIT")
    conn.execute(sqlalchemy.text("""
        CREATE TABLE Lecturer (
            Lecturer_ID  INTEGER      PRIMARY KEY,
            First_Name   VARCHAR(255) NOT NULL,
            Last_Name    VARCHAR(255) NOT NULL,
            Gender       VARCHAR(8)   NOT NULL
        )
    """))
    conn.execute(sqlalchemy.text("""
        CREATE TABLE Student (
            Student_ID   INTEGER      PRIMARY KEY,
            First_Name   VARCHAR(255) NOT NULL,
            Last_Name    VARCHAR(255) NOT NULL,
            Gender       VARCHAR(8)   NOT NULL,
            Student_Type VARCHAR(32)  NOT NULL,
            Age_Group    VARCHAR(32)  NOT NULL,
            Age          INTEGER      NOT NULL
        )
    """))
    conn.execute(sqlalchemy.text("""
        CREATE TABLE Module (
            Module_ID  INTEGER      PRIMARY KEY,
            Title      VARCHAR(255) NOT NULL,
            Runs_In    VARCHAR(32)  NOT NULL,
            Level      CHAR(1)      NOT NULL,
            Credits    INTEGER      NOT NULL
        )
    """))
    conn.execute(sqlalchemy.text("""
        CREATE TABLE Grade (
            Grade_ID         INTEGER     PRIMARY KEY IDENTITY(1,1),
            Last_Try         DATE        NOT NULL,
            Module_ID        INTEGER     NOT NULL,
            Lecturer_ID      INTEGER     NOT NULL,
            Student_ID       INTEGER     NOT NULL,
            Module_Positive  VARCHAR(4)  NOT NULL,
            Number_Of_Tries  INTEGER     NOT NULL,
            Number_Of_Fails  INTEGER     NOT NULL,
            Best_Grade       INTEGER     NOT NULL,
            Credits          INTEGER     NOT NULL,
            FOREIGN KEY (Module_ID) REFERENCES Module(Module_ID),
            FOREIGN KEY (Lecturer_ID) REFERENCES Lecturer(Lecturer_ID),
            FOREIGN KEY (Student_ID) REFERENCES Student(Student_ID)
        )
    """))

Nun wird die Exceldatei *Code_University.xlsx* gelesen.
Dafür muss das Python paket *xlrd* mit dem Befehl *pip3 install xlrd --upgrade* installiert werden.
Es liest das alte Excelformat (xls) ein.

In [3]:
# pip3 install xlrd --upgrade
with engine.connect() as conn:
    for sheet in ["Lecturer", "Student", "Module", "Grade"]:
        data = pd.read_excel("Code_University.xlsx", sheet_name=sheet)
        # Drop unnamed columns
        data = data.loc[:,~data.columns.str.startswith('Unnamed')]
        data.to_sql(sheet, conn, if_exists="append", index=False)
        conn.commit()
